# Deep Architectures
## Code snippets created by [Santiago Pascual](https://github.com/santi-pdp) for the Postgraduate Course on [Artificial Intelligence with Deep Learning](https://github.com/upcschool-ai/2019-spring). [UPC School](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) 2019.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
NUM_INPUTS=100
HIDDEN_SIZE=1024
NUM_OUTPUTS=20

### 1. Logistic Regresion

In [0]:
lor = nn.Sequential(
    nn.Linear(NUM_INPUTS, 1),
    nn.Sigmoid()
)

### 2.Linear Regresion

In [0]:
lir = nn.Sequential(
    nn.Linear(NUM_INPUTS, 1)
)

### 3.  Softmax classifier

In [0]:
smx = nn.Sequential(
    nn.Linear(NUM_INPUTS, NUM_OUTPUTS),
    nn.LogSoftmax(dim=1)
)

### 4. MultiLayer Perceptron

In [0]:
NUM_INPUTS=100
HIDDEN_SIZE=1024
NUM_OUTPUTS=20

mlp = nn.Sequential(
    nn.Linear(NUM_INPUTS, HIDDEN_SIZE),
    nn.Tanh(),
    nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE),
    nn.Tanh(),
    nn.Linear(HIDDEN_SIZE, NUM_OUTPUTS),
    nn.LogSoftmax(dim=1)
)

### 5. Embedding with fully connected layer

In [0]:
VOCAB_SIZE = 10000
HIDDEN_SIZE=100
# mapping a Vocabulary of size 10.000 to HIDDEN_SIZE projections
emb_1 = nn.Linear(VOCAB_SIZE, HIDDEN_SIZE)

In [0]:
# forward example [10, 10000] tensor
code = [1] + [0] * 9999
# copy 10 times the same code [1 0 0 0 ... 0]
x = torch.FloatTensor([code] * 10)
print('Input x tensor size: ', x.size())
y = emb_1(x)
print('Output y embedding size: ', y.size())

Input x tensor size:  torch.Size([10, 10000])
Output y embedding size:  torch.Size([10, 100])


### 6. Embedding with Embedding layer

In [0]:
VOCAB_SIZE = 10000
HIDDEN_SIZE=100
# mapping a Vocabulary of size 10.000 to HIDDEN_SIZE projections
emb_2 = nn.Embedding(VOCAB_SIZE, HIDDEN_SIZE)

In [0]:
# Just make a long tensor with zero-index
x = torch.zeros(10, 1).long()
print('Input x tensor size: ', x.size())
y = emb_2(x)
print('Output y embedding size: ', y.size())

Input x tensor size:  torch.Size([10, 1])
Output y embedding size:  torch.Size([10, 1, 100])


### 7. Recurrent Neural Network

In [0]:
NUM_INPUTS = 100
HIDDEN_SIZE = 512
NUM_LAYERS = 1
# define a recurrent layer
rnn = nn.RNN(NUM_INPUTS, HIDDEN_SIZE, num_layers=NUM_LAYERS)

In [0]:
SEQ_LEN = 100
x = torch.randn(SEQ_LEN, 1, NUM_INPUTS)
print('Input tensor size [seq_len, bsize, hidden_size]: ', x.size())
ht, state = rnn(x, None)
print('Output tensor h[t] size [seq_len, bsize, hidden_size]: ', ht.size())

Input tensor size [seq_len, bsize, hidden_size]:  torch.Size([100, 1, 100])
Output tensor h[t] size [seq_len, bsize, hidden_size]:  torch.Size([100, 1, 512])


In [0]:
NUM_INPUTS = 100
HIDDEN_SIZE = 512
NUM_LAYERS = 1
# define a recurrent layer, swapping batch and time axis
rnn = nn.RNN(NUM_INPUTS, HIDDEN_SIZE, num_layers=NUM_LAYERS,
            batch_first=True)

In [0]:
SEQ_LEN = 100
x = torch.randn(1, SEQ_LEN, NUM_INPUTS)
print('Input tensor size [bsize, seq_len, hidden_size]: ', x.size())
ht, state = rnn(x, None)
print('Output tensor h[t] size [bsize, seq_len, hidden_size]: ', ht.size())

Input tensor size [bsize, seq_len, hidden_size]:  torch.Size([1, 100, 100])
Output tensor h[t] size [bsize, seq_len, hidden_size]:  torch.Size([1, 100, 512])


In [0]:
# let's check ht and state sizes
print('ht size: ', ht.size())
print('state size: ', state.size())

ht size:  torch.Size([1, 100, 512])
state size:  torch.Size([1, 1, 512])


In [0]:
NUM_INPUTS = 100
NUM_OUTPUTS = 10
HIDDEN_SIZE = 512
SEQ_LEN = 100
NUM_LAYERS = 1
# define a recurrent layer, swapping batch and time axis and connect
# an FC layer as an output layer to build a full network
rnn = nn.RNN(NUM_INPUTS, HIDDEN_SIZE, num_layers=NUM_LAYERS,
            batch_first=True)
fc = nn.Sequential(
    nn.Linear(HIDDEN_SIZE, NUM_OUTPUTS),
    nn.LogSoftmax(dim=2)
)

x = torch.randn(1, SEQ_LEN, NUM_INPUTS)
print('Input tensor size x: ', x.size())
ht, state = rnn(x, None)
print('Hidden tensor size ht: ', ht.size())
y = fc(ht)
print('Output tensor y size: ', y.size())


Input tensor size x:  torch.Size([1, 100, 100])
Hidden tensor size ht:  torch.Size([1, 100, 512])
Output tensor y size:  torch.Size([1, 100, 10])


### 8. LSTM Recurrent Neural Network

In [0]:
lstm = nn.LSTM(NUM_INPUTS, HIDDEN_SIZE, num_layers=NUM_LAYERS,
              batch_first=True)
x = torch.randn(1, SEQ_LEN, NUM_INPUTS)
print('Input tensor size x: ', x.size())
ht, states = lstm(x, None)
hT, cT = states[0], states[1]
print('Output tensor ht size: ', ht.size())
print('Last state h[T]: ', hT.size())
print('Cell state c[T]: ', cT.size())

Input tensor size x:  torch.Size([1, 100, 100])
Output tensor ht size:  torch.Size([1, 100, 512])
Last state h[T]:  torch.Size([1, 1, 512])
Cell state c[T]:  torch.Size([1, 1, 512])


### 9. Convolutional Neural Network

In [0]:
NUM_CHANNELS_IN = 1
HIDDEN_SIZE = 1024
KERNEL_WIDTH = 3
# Build a one-dimensional convolutional neural layer
conv1d = nn.Conv1d(NUM_CHANNELS_IN, HIDDEN_SIZE, KERNEL_WIDTH)

In [0]:
SEQ_LEN = 8
x = torch.randn(1, NUM_CHANNELS_IN, SEQ_LEN)
print('Input tensor size x: ', x.size())
y = conv1d(x)
print('Output tensor y size: ', y.size())

Input tensor size x:  torch.Size([1, 1, 8])
Output tensor y size:  torch.Size([1, 1024, 6])


In [0]:
NUM_CHANNELS_IN = 1
HIDDEN_SIZE = 1024
KERNEL_WIDTH = 3
PADDING = KERNEL_WIDTH // 2 # = 1
# Build a one-dimensional convolutional neural layer
conv1d = nn.Conv1d(NUM_CHANNELS_IN, HIDDEN_SIZE, KERNEL_WIDTH, 
                   padding=PADDING)

SEQ_LEN = 8
x = torch.randn(1, NUM_CHANNELS_IN, SEQ_LEN)
print('Input tensor size x: ', x.size())
y = conv1d(x)
print('Output tensor y size: ', y.size())

Input tensor size x:  torch.Size([1, 1, 8])
Output tensor y size:  torch.Size([1, 1024, 8])


In [0]:
NUM_CHANNELS_IN = 1
HIDDEN_SIZE = 1024
KERNEL_WIDTH = 3
# Build a one-dimensional convolutional neural layer
conv1d = nn.Conv1d(NUM_CHANNELS_IN, HIDDEN_SIZE, KERNEL_WIDTH)
                   
SEQ_LEN = 8
PADDING = KERNEL_WIDTH - 1 # = 2
x = torch.randn(1, NUM_CHANNELS_IN, SEQ_LEN)
print('Input tensor x size: ', x.size())
xpad = F.pad(x, (PADDING, 0))
print('Input tensor after padding xpad size: ', xpad.size())
y = conv1d(xpad)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 1, 8])
Input tensor after padding xpad size:  torch.Size([1, 1, 10])
Output tensor y size:  torch.Size([1, 1024, 8])


### 10. Convolutional Neural Network as an MLP

In [0]:
NUM_INPUTS = 100
HIDDEN_SIZE = 1024
NUM_OUTPUTS= 20
# MLP as a CNN
mlp = nn.Sequential(
    nn.Conv1d(NUM_INPUTS, HIDDEN_SIZE, 1),
    nn.Tanh(),
    nn.Conv1d(HIDDEN_SIZE, HIDDEN_SIZE, 1),
    nn.Tanh(),
    nn.Conv1d(HIDDEN_SIZE, NUM_OUTPUTS, 1),
    nn.LogSoftmax(dim=1)
)

x = torch.randn(1, 100, 1)
print('Input tensor x size: ', x.size())
y = mlp(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 100, 1])
Output tensor y size:  torch.Size([1, 20, 1])


### 11. Deconvolutional Neural Network

In [0]:
NUM_CHANNELS_IN = 1
HIDDEN_SIZE = 1
KERNEL_WIDTH = 8
STRIDE = 4

deconv = nn.ConvTranspose1d(NUM_CHANNELS_IN, HIDDEN_SIZE, KERNEL_WIDTH,
                            stride=STRIDE)

SEQ_LEN = 2
y = torch.randn(1, NUM_CHANNELS_IN, SEQ_LEN)
print('Input tensor y size: ', y.size())
x = deconv(y)
print('Output (interpolated) tensor x size: ', x.size())

Input tensor y size:  torch.Size([1, 1, 2])
Output (interpolated) tensor x size:  torch.Size([1, 1, 12])


### 12. Quasi Recurrent Neural Network

In [0]:
class fQRNNLayer(nn.Module):
  
  def __init__(self, num_inputs, num_outputs,
              kwidth=2):
    super().__init__()
    self.num_inputs = num_inputs
    self.num_outputs = num_outputs
    self.kwidth = kwidth
    # double feature maps for zt and ft predictions with same conv layer
    self.conv = nn.Conv1d(num_inputs, num_outputs * 2, kwidth)
    
  def forward(self, x, state=None):
    # x is [bsz, seq_len, num_inputs]
    # state is [bsz, num_outputs] dimensional
    # ---------- FEED FORWARD PART
    # inference convolutional part
    # transpose x axis first to work with CNN layer
    x = x.transpose(1, 2)
    pad = self.kwidth - 1
    xp = F.pad(x, (pad, 0))
    conv_h = self.conv(xp)
    # split convolutional layer feature maps into zt (new state
    # candidate) and forget activation ft
    zt, ft = torch.chunk(conv_h, 2, dim=1)
    # Convert forget gate into actual forget
    ft = torch.sigmoid(ft)
    # Convert zt into actual non-linear response
    zt = torch.tanh(zt)
    # ---------- SEQUENTIAL PART
    # iterate through time now to make pooling
    seqlen = ft.size(2)
    if state is None:
      # create the zero state
      ht_1 = torch.zeros(ft.size(0), self.num_outputs, 1)
    else:
      # add the dim=2 to match 3D tensor shape
      ht_1 = state.unsqueeze(2)
    zts = torch.chunk(zt, zt.size(2), dim=2)
    fts = torch.chunk(ft, ft.size(2), dim=2)
    hts = []
    for t in range(seqlen):
      ht = ht_1 * fts[t] + (1 - fts[t]) * zts[t]
      # transpose time, channels dims again to match RNN-like shape
      hts.append(ht.transpose(1, 2))
      # re-assign h[t-1] now
      ht_1 = ht
    # convert hts list into a 3D tensor [bsz, seq_len, num_outputs]
    hts = torch.cat(hts, dim=1)
    return hts, ht_1.squeeze(2)
      
    
      
fqrnn = fQRNNLayer(1, 100, 2)
x = torch.randn(1, 10, 1)
ht, state = fqrnn(x)
print('ht size: ', ht.size())
print('state size: ', state.size())

ht size:  torch.Size([1, 10, 100])
state size:  torch.Size([1, 100])


### 13. AlexNet classifier

In [0]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [0]:
alexnet = AlexNet()
x = torch.randn(1, 3, 224, 224)
print('Input tensor x size: ', x.size())
y = alexnet(x)
print('Output tensor y size: ', y.size())


Input tensor x size:  torch.Size([1, 3, 224, 224])
Output tensor y size:  torch.Size([1, 1000])


### 14. Residual connections

In [0]:
class ResLayer(nn.Module):
  
  def __init__(self, num_inputs):
    super().__init__()
    self.num_inputs = num_inputs
    num_outputs = num_inputs
    self.num_outputs = num_outputs
    self.conv1 = nn.Sequential(
        nn.Conv2d(num_inputs, num_outputs, 3, padding=1),
        nn.BatchNorm2d(num_outputs),
        nn.ReLU(inplace=True)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(num_outputs, num_outputs, 3, padding=1),
        nn.BatchNorm2d(num_outputs),
        nn.ReLU(inplace=True)
    )
    self.out_relu = nn.ReLU(inplace=True)
    
  def forward(self, x):
    # non-linear processing trunk
    conv1_h = self.conv1(x)
    conv2_h = self.conv2(conv1_h)
    # output is result of res connection + non-linear processing
    y = self.out_relu(x + conv2_h)
    return y
    
x = torch.randn(1, 64, 100, 100)
print('Input tensor x size: ', x.size())
reslayer = ResLayer(64)
y = reslayer(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 64, 100, 100])
Output tensor y size:  torch.Size([1, 64, 100, 100])


### 15. Auto-Encoder Network

In [0]:
class AE(nn.Module):
    def __init__(self, num_inputs=784):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(num_inputs, 400),
            nn.ReLU(inplace=True),
            nn.Linear(400, 400),
            nn.ReLU(inplace=True),
            nn.Linear(400, 20)
        )
        self.decoder = nn.Sequential(
            nn.Linear(20, 400),
            nn.ReLU(inplace=True),
            nn.Linear(400, 400),
            nn.ReLU(inplace=True),
            nn.Linear(400, num_inputs)
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

      
ae = AE(784)
x = torch.randn(10, 784)
print('Input tensor x size: ', x.size())
y = ae(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([10, 784])
Output tensor y size:  torch.Size([10, 784])


### 16. Variational Auto-Encoder Network

In [0]:
# from https://github.com/pytorch/examples/blob/master/vae/main.py
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

vae = VAE()
x = torch.randn(10, 784)
print('Input tensor x size: ', x.size())
y, mu, logvar = vae(x)
print('Input tensor y size: ', y.size())
print('Mean tensor mu size: ', mu.size())
print('Covariance tensor logvar size: ', logvar.size())

Input tensor x size:  torch.Size([10, 784])
Input tensor y size:  torch.Size([10, 784])
Mean tensor mu size:  torch.Size([10, 20])
Covariance tensor logvar size:  torch.Size([10, 20])


### 17. Deep Convolutional Auto-Encoder with skip connections (SEGAN G)

In [0]:
class DownConv1dBlock(nn.Module):
  
  def __init__(self, ninp, fmap, kwidth, stride):
    super().__init__()
    assert stride > 1, stride
    self.kwidth = kwidth
    self.conv = nn.Conv1d(ninp, fmap, kwidth, stride=stride)
    self.act = nn.ReLU(inplace=True)
  
  def forward(self, x):
    # calculate padding with stride > 1
    pad_left = self.kwidth // 2 - 1
    pad_right = self.kwidth // 2
    xp = F.pad(x, (pad_left, pad_right))
    y = self.act(self.conv(xp))
    return y

block = DownConv1dBlock(1, 1, 31, 4)
x = torch.randn(1, 1, 4000)
print('Input tensor x size: ', x.size())
y = block(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 1, 4000])
Output tensor y size:  torch.Size([1, 1, 1000])


In [0]:
class UpConv1dBlock(nn.Module):
  
  def __init__(self, ninp, fmap, kwidth, stride, act=True):
    super().__init__()
    assert stride > 1, stride
    self.kwidth = kwidth
    pad = max(0, (stride - kwidth) // -2)
    self.deconv = nn.ConvTranspose1d(ninp, fmap, kwidth,
                                    stride=stride,
                                    padding=pad)
    if act:
      self.act = nn.ReLU(inplace=True)
  
  def forward(self, x):
    h = self.deconv(x)
    if self.kwidth % 2 != 0:
      # drop last item for shape compatibility with TensorFlow deconvs
      h = h[:, :, :-1]
    if hasattr(self, 'act'):
      y = self.act(h)
    else:
      y = h
    return y

block = UpConv1dBlock(1, 1, 31, 4)
x = torch.randn(1, 1, 1000)
print('Input tensor x size: ', x.size())
y = block(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 1, 1000])
Output tensor y size:  torch.Size([1, 1, 4000])


In [0]:
class Conv1dGenerator(nn.Module):
  
  def __init__(self, enc_fmaps=[64, 128, 256, 512], kwidth=31,
               pooling=4):
    super().__init__()
    self.enc = nn.ModuleList()
    ninp = 1
    for enc_fmap in enc_fmaps:
      self.enc.append(DownConv1dBlock(ninp, enc_fmap, kwidth, pooling))
      ninp = enc_fmap
    
    self.dec = nn.ModuleList()
    # revert encoder feature maps
    dec_fmaps = enc_fmaps[::-1][1:] + [1]
    act = True
    for di, dec_fmap in enumerate(dec_fmaps, start=1):
      if di >= len(dec_fmaps):
        # last decoder layer has no activation
        act = False
      self.dec.append(UpConv1dBlock(ninp, dec_fmap, kwidth, pooling, act=act))
      ninp = dec_fmap
  
  def forward(self, x):
    skips = []
    h = x
    for ei, enc_layer in enumerate(self.enc, start=1):
      h = enc_layer(h)
      if ei < len(self.enc):
        skips.append(h)
    # now decode
    
    for di, dec_layer in enumerate(self.dec, start=1):
      if di > 1:
        # sum skip connection
        skip_h = skips.pop(-1)
        h = h + skip_h
      h = dec_layer(h)
    y = h
    return y
      
G = Conv1dGenerator()
x = torch.randn(1, 1, 8192)
print('Input tensor x size: ', x.size())
y = G(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 1, 8192])
Output tensor y size:  torch.Size([1, 1, 8192])


### 18. DCGAN G and D

In [0]:
# from https://github.com/pytorch/examples/blob/master/dcgan/main.py
class Generator(nn.Module):
    def __init__(self, nc=3):
        super().__init__()
        nz = 100
        ngf = 64
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
      return self.main(input)

z = torch.randn(1, 100, 1, 1)
print('Input tensor z size: ', z.size())
G = Generator()
x = G(z)
print('Output tensor x size: ', x.size())

Input tensor z size:  torch.Size([1, 100, 1, 1])
Output tensor x size:  torch.Size([1, 3, 64, 64])


In [0]:
class Discriminator(nn.Module):
    def __init__(self, nc=3):
        super(Discriminator, self).__init__()
        ndf = 64
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)
      
      
x = torch.randn(1, 3, 64, 64)
print('Input tensor x size: ', x.size())
D = Discriminator()
y = D(x)
print('Output tensor y size: ', y.size())

Input tensor x size:  torch.Size([1, 3, 64, 64])
Output tensor y size:  torch.Size([1, 1, 1, 1])
